# Advent of Code

## 2023-012-020
## 2023 020

https://adventofcode.com/2023/day/20

In [ ]:
import collections

class Module:
    def __init__(self, name, module_type=None):
        self.name = name
        self.type = module_type
        self.outputs = []
        self.inputs = []
        self.state = 'off'  # Only relevant for flip-flop modules (%)
        self.memory = {}  # Only relevant for conjunction modules (&)
        
    def add_output(self, module):
        self.outputs.append(module)
        
    def add_input(self, module):
        self.inputs.append(module)
        
    def receive_pulse(self, pulse_type):
        pass
        
class FlipFlop(Module):
    def __init__(self, name):
        super().__init__(name, '%')
    
    def receive_pulse(self, pulse_type):
        if pulse_type == 'low':
            if self.state == 'off':
                self.state = 'on'
                return 'high'
            else:
                self.state = 'off'
                return 'low'
        return None

class Conjunction(Module):
    def __init__(self, name):
        super().__init__(name, '&')
    
    def receive_pulse(self, pulse_type):
        if self.name not in self.memory:
            self.memory[self.name] = 'low'
        self.memory[self.name] = pulse_type
        
        if all(value == 'high' for value in self.memory.values()):
            return 'low'
        else:
            return 'high'

class Broadcaster(Module):
    def __init__(self, name):
        super().__init__(name)
    
    def receive_pulse(self, pulse_type):
        # A broadcaster sends the pulse to all its outputs
        return [pulse_type for _ in self.outputs]

def parse_input(input_data):
    modules = {}
    lines = input_data.strip().splitlines()
    
    # Parse each line
    for line in lines:
        left, right = line.split('->')
        outputs = right.split(',')
        
        # Extract module name and type (if any)
        left_name = left.strip()
        module_type = None
        
        if left_name.startswith('%'):
            module_type = '%'
            left_name = left_name[1:]
        elif left_name.startswith('&'):
            module_type = '&'
            left_name = left_name[1:]
        
        if left_name not in modules:
            if module_type == '%':
                modules[left_name] = FlipFlop(left_name)
            elif module_type == '&':
                modules[left_name] = Conjunction(left_name)
            else:
                modules[left_name] = Broadcaster(left_name)
        
        # Connect outputs to destination modules
        for output in outputs:
            output = output.strip()
            if output not in modules:
                modules[output] = Broadcaster(output)
            modules[left_name].add_output(modules[output])
            modules[output].add_input(modules[left_name])
    
    return modules

def simulate_pulses(modules, num_iterations=1000):
    # The initial pulse when the button is pushed
    low_pulses_sent = 0
    high_pulses_sent = 0
    
    # Simulate button presses
    for _ in range(num_iterations):
        # Initial low pulse sent to broadcaster
        pulse_queue = [('broadcaster', 'low')]
        
        # Process pulses
        while pulse_queue:
            current_module_name, pulse_type = pulse_queue.pop(0)
            current_module = modules[current_module_name]
            
            # The module processes the pulse and generates its output
            if isinstance(current_module, FlipFlop):
                new_pulse = current_module.receive_pulse(pulse_type)
                if new_pulse:
                    pulse_queue.append((current_module.outputs[0].name, new_pulse))
                    if new_pulse == 'low':
                        low_pulses_sent += 1
                    else:
                        high_pulses_sent += 1
            elif isinstance(current_module, Conjunction):
                new_pulse = current_module.receive_pulse(pulse_type)
                if new_pulse:
                    for output in current_module.outputs:
                        pulse_queue.append((output.name, new_pulse))
                    if new_pulse == 'low':
                        low_pulses_sent += 1
                    else:
                        high_pulses_sent += 1
            elif isinstance(current_module, Broadcaster):
                for output in current_module.outputs:
                    pulse_queue.append((output.name, pulse_type))
                    if pulse_type == 'low':
                        low_pulses_sent += 1
                    else:
                        high_pulses_sent += 1

    return low_pulses_sent, high_pulses_sent

# Read the input data
input_data = '''&jc -> vq, mf, bv, pk, nc, sv, rl
%bj -> zc
%km -> jc, fc
%vr -> xq, qq
&ft -> xm
&jz -> xm
&fj -> jz, bj, mr, tp, ql, kf
%pt -> qq, pf
%zc -> qv, fj
%sr -> vr
%tr -> qq
%lq -> fj, ql
%qv -> kf, fj
%sn -> nk
%jd -> jc, gm
%tp -> bj, fj
%mp -> vm, nn
broadcaster -> pt, tp, gv, bv
%qh -> fj, nm
%gv -> vm, kq
%xt -> qq, lh
%nm -> fj, js
%hj -> ch
%mb -> vm, qg
%gr -> fj, qh
%js -> fj
%rl -> nc
&qq -> sr, pt, ch, lh, hj, pf, ft
%bv -> jc, mf
%nv -> mb
&xm -> rx
%nc -> km
&sv -> xm
%ql -> gr
%vn -> jc
%hv -> qq, hj
&vm -> ng, hz, sn, gv, nv
%rr -> qq, tr
%vv -> jc, vn
&ng -> xm
%nn -> cv, vm
%ch -> xt
%mr -> lq
%cv -> vm
%fc -> vv, jc
%pf -> hv
%pk -> vq
%vq -> jd
%kf -> mr
%mf -> pk
%qg -> vm, sn
%nk -> vk, vm
%hz -> mp
%kq -> vm, nv
%lh -> sr
%gm -> jc, rl
%vk -> vm, hz
%xq -> qq, rr'''

# Parse the input
modules = parse_input(input_data)

# Simulate pulses for 1000 iterations
low_pulses, high_pulses = simulate_pulses(modules, 1000)

# Calculate the result
result = low_pulses * high_pulses
print(result)